In [1]:
from DolphinApi.config import api_url, api_user_auth, api_pass_auth, portofolio_label, start_period, end_period

In [2]:
import requests
import json
import pandas as pd

In [3]:
class DolphinApi:
    def __init__(self):
        self.url = api_url
        self.auth = (api_user_auth, api_pass_auth)
        self.portofolio_label = portofolio_label
        try:
            self.currency_table = pd.read_csv("currency_table.csv", index_col=0)
        except FileNotFoundError: 
            self.currency_table = self.__get_currency_rate__()
            self.currency_table.to_csv("currency_table.csv")
        try:
            self.operations_table = pd.read_csv("operations_table.csv", index_col=0)
        except FileNotFoundError: 
            self.operations_table = self.__get_operations_table__()
            self.operations_table.to_csv("operations_table.csv")

    def get(self, endpointApi, date=None, full_response=False):
        payload = {'date': date, 'fullResponse': full_response}
        res = requests.get(self.url + endpointApi,
                           params=payload,
                           auth=self.auth)
        return res.content.decode('utf-8')

    def put(self, endpointApi, content):
        res = requests.put(url=self.url + endpointApi,
                          data=json.dumps(content),
                          auth=self.auth,
                          headers = {"content-type": "application/json"})
        return res.content.decode('utf-8')

    def post(self, endpointApi, content):
        res = requests.post(url=self.url + endpointApi,
                    data=json.dumps(content),
                    auth=self.auth,
                    headers = {"content-type": "application/json"})
        return res.content.decode('utf-8')

    def __get_currency_rate__(self):
        d = []
        arr = json.loads(self.get('currency'))
        for currency in arr:
            currency_id = currency.get('id')
            rate = self.get('currency/rate/{}/to/EUR'.format(currency_id))
            if len(rate) != 0:
                d.append([currency_id, (json.loads(rate)['rate']['value']).replace(',', '.')])
        return pd.DataFrame(d, columns=['currency', 'rate'])

    def __get_operations_table__(self):
        data = self.get('ratio')
        return pd.read_json(data)

api = DolphinApi()

In [4]:
from datetime import datetime
import pandas as pd
import numpy as np
import json


In [5]:
def to_eur(src_value):
    value, src_currency = src_value.split(' ')
    rate = api.currency_table[api.currency_table['currency']
                              == src_currency]['rate'].values[0]
    return float(value.replace(',', '.')) * float(rate)


def convert_type(df):
    for col in df.columns:
        convert_values = []
        for elt in df[col]:
            if elt is np.nan:
                convert_values.append(np.nan)
                continue
            elt_type, elt_value = elt['type'], elt['value']
            if elt_type in ['asset', 'int32', 'int64']:
                elt_value = int(elt_value)
            elif elt_type in ['double', 'percent']:
                elt_value = float(elt_value.replace(',', '.'))
            elif elt_type == 'date':
                elt_value = datetime.strptime(elt_value, '%Y-%m-%d').date()
            elif elt_type == 'currency_value':
                elt_value = to_eur(elt_value)
            elif elt_type == 'boolean':
                elt_value = json.loads(elt_value)
            convert_values.append(elt_value)
        df[col] = convert_values
    return df


def get_asset_full_info(id_):
    data = api.get('asset/')
    asset = convert_type(pd.read_json(data))
    asset[asset['ASSET_DATABASE_ID'] == id_].to_csv("test.csv")
    return asset[asset['ASSET_DATABASE_ID'] == id_]

In [6]:
def get_assets_ids(date):
    cols = ["columns=ASSET_DATABASE_ID", "columns=LABEL",
            "columns=TYPE", "columns=LAST_CLOSE_VALUE_IN_CURR",
            "columns=CURRENCY", "columns=MIN_BUY_AMOUNT",
            "columns=asset_fund_info_decimalisation"]
    endpointApi = "asset?{}&date={}".format("&".join(cols), date)
    data = pd.read_json(api.get(endpointApi))
    assets = convert_type(data)
    assets = assets[(assets['LAST_CLOSE_VALUE_IN_CURR'].notna())
                    & (assets['TYPE'] != 'PORTFOLIO')].reset_index()
    assets['MIN_BUY_AMOUNT'] = assets['MIN_BUY_AMOUNT'].fillna(value=1)
    assets['asset_fund_info_decimalisation'] = assets['asset_fund_info_decimalisation'].fillna(
        value=0)
    return assets[['ASSET_DATABASE_ID', 'CURRENCY', 'MIN_BUY_AMOUNT', 'asset_fund_info_decimalisation', "TYPE", "LAST_CLOSE_VALUE_IN_CURR"]]


def get_type_table():
    try:
        type_table = pd.read_csv("type_table.csv", index_col=0)
        return type_table
    except FileNotFoundError:
        type_table = get_assets_ids(start_period)
        type_table[["ASSET_DATABASE_ID", "TYPE"]].to_csv("type_table.csv")
        return type_table[["ASSET_DATABASE_ID", "TYPE"]]


def get_price_table():
    try:
        type_table = pd.read_csv("price_table.csv", index_col=0)
        return type_table
    except FileNotFoundError:
        type_table = get_assets_ids(start_period)
        type_table[["ASSET_DATABASE_ID", "LAST_CLOSE_VALUE_IN_CURR"]].to_csv(
            "price_table.csv")
        return type_table[["ASSET_DATABASE_ID", "LAST_CLOSE_VALUE_IN_CURR"]]


def get_type(id_):
    type_table = get_type_table()
    return type_table[type_table['ASSET_DATABASE_ID'] == id_].values[0, 1]


def get_types(ids):
    type_table = get_type_table()
    return type_table[type_table['ASSET_DATABASE_ID'].isin(ids)].values[:, 1]


def get_types_ids(ids, types):
    type_table = get_type_table()
    return [i for i, id_ in enumerate(ids) if type_table[type_table['ASSET_DATABASE_ID'] == id_].values[0, 1] in types]


def select_type(type_list):
    table_type = get_type_table()
    table_type = table_type[table_type.TYPE.isin(type_list)].ASSET_DATABASE_ID
    return table_type.values


def get_price(id_):
    price_table = get_price_table()
    return price_table[price_table['ASSET_DATABASE_ID'] == id_].values[0, 1]


def get_prices(ids):
    price_table = get_price_table()
    return price_table[price_table['ASSET_DATABASE_ID'].isin(ids)].values[:, 1]


def get_quote(id_, start, end):
    data = api.get('asset/{}/quote?start_date={}&end_date={}'
                   .format(id_, start, end))
    return convert_type(pd.read_json(data))


def process_val(close, asset_currency, asset_min_buy, decimalisation):
    return pow(10, -decimalisation) * (asset_min_buy or 1) * to_eur(str(close) + ' ' + asset_currency)


def get_quote_matrixes(start, end):
    try:
        all_closes = pd.read_csv("all_closes.csv", index_col=0)
        all_returns = pd.read_csv("all_returns.csv", index_col=0)
        return (all_closes, all_returns)
    except FileNotFoundError:
        assets = get_assets_ids(start)
        cur = assets.values[0]
        all_assets = get_quote(cur[0], start, end)
        close_matrix = all_assets[['close']].set_index(all_assets.date)
        close_matrix['close'] = close_matrix['close'].apply(
            lambda x: process_val(x, cur[1], cur[2], cur[3]))
        close_matrix.columns = ['{}'.format(cur[0])]
        return_matrix = all_assets[['return']].set_index(all_assets.date)
        return_matrix.columns = ['{}'.format(cur[0])]
        for i in range(1, len(assets)):
            cur = assets.values[i]
            all_assets = get_quote(cur[0], start, end)
            if 'close' in all_assets:
                cur_close = all_assets[['close']].set_index(all_assets.date)
                cur_close['close'] = cur_close['close'].apply(
                    lambda x: process_val(x, cur[1], cur[2], cur[3]))
                cur_close.columns = ['{}'.format(cur[0])]
                close_matrix = pd.concat(
                    [close_matrix, cur_close], axis=1, sort=False)
            if 'return' in all_assets:
                cur_return = all_assets[['return']].set_index(all_assets.date)
                cur_return.columns = ['{}'.format(cur[0])]
                return_matrix = pd.concat(
                    [return_matrix, cur_return], axis=1, sort=False)

        all_closes = close_matrix.sort_index().fillna(method='pad')
        all_returns = return_matrix.sort_index().fillna(method='pad')
        all_closes.to_csv("all_closes.csv")
        all_returns.to_csv("all_returns.csv")
        return (all_closes, all_returns)


In [7]:
def get_portfolio(id_):
    data = api.get('portfolio/{}/dyn_amount_compo'.format(id_))
    portfolio = pd.read_json(data)
    return portfolio


def get_portfolio_IDs():
    cols = ["columns=TYPE", "columns=ASSET_DATABASE_ID", "columns=LABEL"]
    endpointApi = "asset?{}&date={}".format("&".join(cols), start_period)
    data = pd.read_json(api.get(endpointApi))
    assets = convert_type(data)
    assets = assets[(assets['TYPE'] == 'PORTFOLIO')].reset_index()
    return assets
    portfolio_id = assets.loc[
        (assets['TYPE'] == 'PORTFOLIO') &
        (assets['LABEL'] == api.portofolio_label)]['ASSET_DATABASE_ID'].values[0]
    return int(portfolio_id)


def get_epita_portfolio_id():
    df_IDs = get_portfolio_IDs()
    return int(df_IDs.loc[df_IDs['LABEL'] == 'EPITA_PTF_4']['ASSET_DATABASE_ID'])


def get_epita_portfolio():
    epita_portfolio_id = get_epita_portfolio_id()
    return get_portfolio(epita_portfolio_id)


def get_assets_portfolio(portfolio, date):
    if date not in portfolio['values']:
        return np.NaN
    return portfolio['values'][date]


def put_portfolio(portfolio_id, df_portfolio, assets):
    label = df_portfolio['label'][0]
    currency = df_portfolio['currency'][0]
    type_ = df_portfolio['type'][0]
    date = '2016-06-01'
    form = '{{"asset":{{"asset": {}, "quantity": {}}}}},'
    assets = ''.join([form.format(int(assets.iloc[i, 0]),
                                  int(assets.iloc[i, 1]))
                      for i in range(len(assets))])[:-1]
    form = '{{"label": "{}", "currency": {{"code": "{}"}}, "type": "{}", "values": {{"{}": [{}]}}}}'
    res = form.format(label, currency, type_, date, assets)
    form = 'portfolio/{}/dyn_amount_compo'
    api.put(form.format(portfolio_id), json.loads(res))


In [8]:
from pyswarm import pso
import scipy.optimize as optimize

In [9]:
def stock_constraint(x, price_mat, stock_ids):
    complete_price = np.dot(x, price_mat)
    stocks_price = np.dot(x[stock_ids], price_mat[stock_ids])
    return stocks_price / complete_price

def nav_constraint(x, price_mat, stock_ids):
    complete_price = np.dot(x, price_mat)
    stocks_price = x[stock_ids] * price_mat[stock_ids]
    nav_percent = stocks_price / complete_price
    return np.all(nav_percent) < 0.1 and np.all(nav_percent) > 0.01


def opti_min_func(weights, assets_id, return_matrix, cov_matrix):
    """
    Function to calculate Sharpe ratio
    """
    weights = [w / sum(weights) for w in weights]
    weights = np.matrix(weights)
    port_return = np.round(np.sum(weights * return_matrix.T) * 1274, 2)/5
    port_volacity = np.round(
        np.sqrt(weights * cov_matrix * weights.T) * np.sqrt(1274), 2)/np.sqrt(5)
    sharpe_ratio = (port_return - 0.05) / float(port_volacity)
    print(sharpe_ratio)
    return - sharpe_ratio


def pso_optimise(assets_ids, fast):
    data = get_quote_matrixes(start_period, end_period)[
        1].fillna(method='bfill')
    stock_counter = 1
    return_matrix = []
    cov_input = []

    for i in assets_ids:
        avg_return = data[str(i)].values.mean()
        return_matrix.append(avg_return)
        cov_input.append(data[str(i)].tolist())

    return_matrix = np.matrix(return_matrix)
    cov_input = np.matrix(cov_input)
    cov_matrix = np.cov(cov_input)

    portefolio_id = get_epita_portfolio_id()
    portefolio = get_epita_portfolio()
    nb_assets = len(assets_ids)

    fast_lb = [0] * nb_assets
    lb = [0.01] * nb_assets
    ub = [0.1] * nb_assets

    prices = get_prices(assets_ids)
    stocks = get_types_ids(assets_ids, ["STOCK"])

    constraints = [lambda x, assets_ids, c, d: np.sum(x) - 1,
                   lambda x, assets_ids, c, d: stock_constraint(x, prices, np.array(stocks).astype(int)) - 0.51,
                   lambda x, assets_ids, c, d: nav_constraint(x, prices, np.array(stocks).astype(int)) - 0.5]

    if(not fast):
        xopt, fopt = pso(opti_min_func, lb, ub, ieqcons=[constraints[0]], args=(
            assets_ids, return_matrix, cov_matrix), debug=True, swarmsize=1000, omega=0.9, phip=0.1, phig=0.1, maxiter=20)
    else:
        xopt, fopt = pso(opti_min_func, fast_lb, ub, ieqcons=[constraints[0]], args=(
            assets_ids, return_matrix, cov_matrix), debug=True, swarmsize=100, maxiter=20, minstep=1e-4)

    print(xopt)
    optimal_sharpe_arr = xopt
    return np.array(optimal_sharpe_arr)


def scipy_optimise(assets_ids, fast):
    data = get_quote_matrixes(start_period, end_period)[
        1].fillna(method='bfill')
    stock_counter = 1
    return_matrix = []
    cov_input = []

    for i in assets_ids:
        avg_return = data[str(i)].values.mean()
        return_matrix.append(avg_return)
        cov_input.append(data[str(i)].tolist())

    return_matrix = np.matrix(return_matrix)
    cov_input = np.matrix(cov_input)
    cov_matrix = np.cov(cov_input)

    portefolio_id = get_epita_portfolio_id()
    portefolio = get_epita_portfolio()
    nb_assets = len(assets_ids)

    ws = [1/nb_assets] * nb_assets

    fast_rangeb = tuple((0, 0.1) for i in range(nb_assets))
    rangeb = tuple((0.01, 0.1) for i in range(nb_assets))

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},)

    if(not fast):
        xopt = optimize.minimize(opti_min_func,
                                 ws,
                                 (assets_ids, return_matrix, cov_matrix),
                                 method='SLSQP',
                                 options={'maxiter': 500, 'ftol': 1e-06,
                                          'iprint': 1, 'disp': True, 'eps': 0.1},
                                 bounds=rangeb,
                                 constraints=constraints)
    else:
        xopt = optimize.minimize(opti_min_func,
                                 ws,
                                 (assets_ids, return_matrix, cov_matrix),
                                 method='SLSQP',
                                 options={'maxiter': 500, 'ftol': 1e-07,
                                          'iprint': 1, 'disp': False, 'eps': 0.1},
                                 bounds=fast_rangeb,
                                 constraints=constraints)

    print(xopt)
    optimal_sharpe_arr = xopt.x
    return np.array(optimal_sharpe_arr)


In [11]:
def post_operations(ratios, ids, start, end, bench=None, frequency=None):
    """
    Post request to the API

    Parameters
    ----------
    ratios  : array
        array of the operations to compute
    ids     : array
        array the assets IDs
    start   : date
        format: 'Y-m-d'
    end     : date
        format: 'Y-m-d'
    Returns
    -------
        Dataframe of the response
    """

    payload = {'ratio': ratios,
               'asset': ids,
               'start_date': start,
               'end_date': end,
               'bench': bench,
               'frequency': frequency}
    data = api.post('ratio/invoke', payload)
    data = pd.read_json(data)
    operation = convert_type(data)
    operation = operation.transpose()
    operation.columns = np.array(
        [api.operations_table[api.operations_table.id == i].name.values[0] for i in ratios])
    return operation


def check_constraints(assets_ids, x):
    prices = get_prices(assets_ids)
    stocks = get_types_ids(assets_ids, ["STOCK"])
    print("stock part %:", stock_constraint(x, prices, np.array(stocks).astype(int))*100)
    print("%nav between 0.01 and 0.1:", nav_constraint(x, prices, np.array(stocks).astype(int)))
    print("assets between 15 and 40:", len(assets_ids) > 14 and len(assets_ids) < 41)


def sharping_together(algo_opti):
    stock_ids = select_type(["STOCK"])
    fund_ids = select_type(["ETF FUND", "FUND", "INDEX"])
    portefolio_id = get_epita_portfolio_id()
    portefolio = get_epita_portfolio()

    print("STOCKS")
    stock_part = algo_opti(stock_ids, True)
    df = pd.DataFrame(np.stack((stock_ids, stock_part), axis=-1),
                      columns=["ids", "part"]).sort_values(by="part").values
    stock_ids = df[:, 0][::-1][:40].astype(int)

    print("NOT STOCKS")
    fund_part = algo_opti(fund_ids, True)
    df = pd.DataFrame(np.stack((fund_ids, fund_part), axis=-1),
                      columns=["ids", "part"]).sort_values(by="part").values
    fund_ids = df[:, 0][::-1][:0].astype(int)  # replace 100 by 5 at worst case

    print("REDUCE PART")
    reduced_ids = np.concatenate((stock_ids, fund_ids))
    reduced_part = algo_opti(reduced_ids, True)
    df = pd.DataFrame(np.stack((reduced_ids, reduced_part), axis=-1),
                      columns=["ids", "part"]).sort_values(by="part").values
    final_ids = df[:, 0][::-1][:15].astype(int)

    print("COMPUTE BEST")
    final_part = algo_opti(final_ids, False)

    check_constraints(final_ids, final_part)

    assets_dataframe = pd.DataFrame(
        data={'asset_id': final_ids, 'quantities': final_part * 1000000})

    print(assets_dataframe)
    put_portfolio(portefolio_id, portefolio, assets_dataframe)
    post_operations([12], [portefolio_id], start_period,
                    end_period).values[0, 0]
    print("sharp of portfolio =", post_operations(
        [12], [portefolio_id], start_period, end_period).values[0, 0])

    return final_part


def sharping_stocks(algo_opti):
    stock_ids = select_type(["STOCK"])
    portefolio_id = get_epita_portfolio_id()
    portefolio = get_epita_portfolio()

    print("REDUCE")
    stock_part = algo_opti(stock_ids, True)
    df = pd.DataFrame(np.stack((stock_ids, stock_part), axis=-1),
                      columns=["ids", "part"]).sort_values(by="part").values
    final_ids = df[:, 0][::-1][:40].astype(int)


    print("COMPUTE BEST")
    final_part = algo_opti(final_ids, False)

    check_constraints(final_ids, final_part)

    assets_dataframe = pd.DataFrame(
        data={'asset_id': final_ids, 'quantities': final_part * 1000000})

    print(assets_dataframe)
    put_portfolio(portefolio_id, portefolio, assets_dataframe)
    post_operations([12], [portefolio_id], start_period,
                    end_period).values[0, 0]
    print("sharp of portfolio =", post_operations(
        [12], [portefolio_id], start_period, end_period).values[0, 0])

    return final_part


def get_best_weigth(algo):
    if(algo == "scipy"):
        return sharping_stocks(scipy_optimise)
    elif (algo == "pso"):
        return sharping_stocks(pso_optimise)
    else:
        print("choose an algorithm : 'pso' or 'scipy'")


In [12]:
get_best_weigth("pso")

8382 0.1       ] -2.8208857562305036
Best after iteration 4: [0.01       0.08752781 0.1        0.01       0.01       0.01
 0.1        0.01       0.1        0.01       0.1        0.01
 0.02715449 0.05976229 0.08371301 0.01       0.1        0.01
 0.0919583  0.01       0.1        0.1        0.01       0.1
 0.01       0.03580277 0.01       0.01       0.01       0.01
 0.01       0.01       0.0788246  0.01       0.03822151 0.1
 0.01       0.01       0.03618382 0.1       ] -2.8208857562305036
Best after iteration 5: [0.01       0.08752781 0.1        0.01       0.01       0.01
 0.1        0.01       0.1        0.01       0.1        0.01
 0.02715449 0.05976229 0.08371301 0.01       0.1        0.01
 0.0919583  0.01       0.1        0.1        0.01       0.1
 0.01       0.03580277 0.01       0.01       0.01       0.01
 0.01       0.01       0.0788246  0.01       0.03822151 0.1
 0.01       0.01       0.03618382 0.1       ] -2.8208857562305036
New best for swarm at iteration 6: [0.01       0.099476

array([0.01      , 0.0187791 , 0.1       , 0.01      , 0.01      ,
       0.01      , 0.01      , 0.01      , 0.01121814, 0.01      ,
       0.1       , 0.01      , 0.01      , 0.01      , 0.01      ,
       0.01      , 0.05962713, 0.01784015, 0.01      , 0.01      ,
       0.02756279, 0.04872824, 0.01      , 0.1       , 0.01      ,
       0.01      , 0.01      , 0.01      , 0.01      , 0.01      ,
       0.01      , 0.01      , 0.01453046, 0.02787765, 0.01      ,
       0.01      , 0.07208064, 0.01      , 0.1       , 0.05107299])